<a href="https://colab.research.google.com/github/Elshamysamira/Information-Extraction-and-Retrieval/blob/main/IR_26_05_2024__Semantic_YES_WORKING_streamlit_4%2B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary packages
!pip install datasets
!pip install faiss-cpu

import nltk
import numpy as np
import chardet
import torch
from datasets import Dataset
from transformers import DPRContextEncoder, DPRContextEncoderTokenizer
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
from nltk.tokenize import word_tokenize
from collections import defaultdict
import pandas as pd
from pathlib import Path
import os
import sqlite3
import re
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 14.8 MB/s eta 0:00:00


In [ ]:
nltk.download('punkt')

drive.mount('/content/drive/')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at /content/drive/


In [ ]:
class InvertedIndex:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS inverted_index (
                                word TEXT PRIMARY KEY,
                                document_ids TEXT
                            )''')

    def save_index(self, inverted_index):
        for word, document_ids in inverted_index.items():
            doc_ids_str = ','.join(str(doc_id) for doc_id in document_ids)
            self.cursor.execute("INSERT INTO inverted_index (word, document_ids) VALUES (?, ?)", (word, doc_ids_str))
        self.conn.commit()

    def close_connection(self):
        self.conn.close()

class DocumentTokenizer:
    def __init__(self, documents):
        self.documents = documents

    def tokenize(self):
        tokenized_docs = {}
        for documentID, document_content in enumerate(self.documents):
            try:
                tokens = word_tokenize(document_content)
                tokenized_docs[documentID] = tokens
            except Exception as e:
                print(f"Error processing document {documentID}: {e}")
        return tokenized_docs

class InvertedIndexBuilder:
    def __init__(self, documents):
        self.documents = documents
        self.inverted_index = defaultdict(set)

    def build_index(self):
        for documentID, tokens in self.documents.items():
            for word in tokens:
                self.inverted_index[word.lower()].add(documentID)

    def get_index(self):
        return self.inverted_index

class SearchEngine:
    def __init__(self, index_db_path, documents_mapping):
        self.index_db_path = index_db_path
        self.documents_mapping = documents_mapping

    def lookup_word(self, word):
        conn = sqlite3.connect(self.index_db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT document_ids FROM inverted_index WHERE word=?", (word,))
        result = cursor.fetchone()
        conn.close()

        if result:
            document_ids = set(map(int, result[0].split(',')))
            return document_ids
        else:
            return set()

    def search(self, query):
        query_tokens = word_tokenize(query.lower())
        print(f"Tokenized Query: {query_tokens}")  # Print tokenized query

        document_sets = [self.lookup_word(token) for token in query_tokens]
        common_documents = set.intersection(*document_sets) if document_sets else set()

        if common_documents:
            print(f"Congratulations! The word(s) '{query}' appear together in the following document ID(s): {common_documents}")
            for doc_id in common_documents:
                print(f"Document ID: {doc_id}, Document Name: {self.documents_mapping.get(doc_id, 'Unknown')}")
        else:
            print(f"I'm sorry, the word(s) '{query}' do not appear together in any document.")

class DocumentManager:
    def __init__(self, books_path):
        self.books_path = books_path

    def get_doc_paths(self):
        return [os.path.join(self.books_path, file) for file in os.listdir(self.books_path) if os.path.isfile(os.path.join(self.books_path, file))]

def clean_text(text):
    return re.sub(r'\n\s*\n+', '\n\n', text)

def load_and_clean_documents(doc_paths):
    cleaned_docs = []
    for doc_id, doc_path in enumerate(doc_paths):
        with open(doc_path, 'r', encoding='utf-8', errors='ignore') as file:
            text = file.read()
            cleaned_text = clean_text(text)
            cleaned_docs.append((doc_id, cleaned_text, Path(doc_path).stem))
    return cleaned_docs

# Paths
books_path = '/content/drive/My Drive/Documents'
index_db_path = '/content/drive/My Drive/Documents/inverted_index.db'

# Initialize DocumentManager
doc_manager = DocumentManager(books_path)
doc_paths = doc_manager.get_doc_paths()

# Clean documents and generate embeddings
cleaned_documents = load_and_clean_documents(doc_paths)
cleaned_texts = [doc[1] for doc in cleaned_documents]

# Tokenize documents
doc_tokenizer = DocumentTokenizer(cleaned_texts)
tokenized_docs = doc_tokenizer.tokenize()

# Build inverted index
index_builder = InvertedIndexBuilder(tokenized_docs)
index_builder.build_index()
inverted_index = index_builder.get_index()

# Save inverted index to SQLite
index_db = InvertedIndex(index_db_path)
index_db.save_index(inverted_index)
index_db.close_connection()

# Document mapping
document_mapping = {doc_id: doc_title for doc_id, _, doc_title in cleaned_documents}

# Initialize SearchEngine
search_engine = SearchEngine(index_db_path, document_mapping)
search_engine.search('HateD')
print('\n')
search_engine.search('HateD Applied')
print('\n')
search_engine.search("Can't")
print('\n')
search_engine.search("didn't")
print('\n')
search_engine.search("state-of-the-art")
print('\n')
search_engine.search("Elliott-Fisher")
print('\n')
search_engine.search("Mr.")
print('\n')
search_engine.search("Mr")

connect_db = sqlite3.connect(index_db_path)
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", connect_db)

for table in tables['name']:
    print(f"Table Name: {table}")
    query = "SELECT * FROM inverted_index"
    df = pd.read_sql_query(query, connect_db)
    display(df)
    print('\n')

connect_db.close()

# Semantic Similarity using DPR
torch.set_grad_enabled(False)

# Load the DPR context encoder and tokenizer
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Load and prepare dataset from cleaned documents
def load_paragraphs_for_dpr(cleaned_docs):
    paragraphs = []
    for doc_id, content, doc_title in cleaned_docs:
        for para_id, paragraph in enumerate(content.split('\n\n')):
            paragraphs.append({"doc_id": doc_id, "doc_title": doc_title, "para_id": para_id, "content": paragraph})
    return paragraphs

paragraphs = load_paragraphs_for_dpr(cleaned_documents)

# Print the first few entries of the paragraphs list for inspection
for paragraph in paragraphs[:50]:
    print(paragraph['content'])
    print('[END of paragraph]')  # Add an empty line for better readability

ds = Dataset.from_dict({
    "doc_id": [para["doc_id"] for para in paragraphs],
    "doc_title": [para["doc_title"] for para in paragraphs],
    "para_id": [para["para_id"] for para in paragraphs],
    "content": [para["content"] for para in paragraphs],
})

Tokenized Query: ['hated']
Congratulations! The word(s) 'HateD' appear together in the following document ID(s): {1}
Document ID: 1, Document Name: Dumbells of Business by Louis Custer Martin Reed


Tokenized Query: ['hated', 'applied']
I'm sorry, the word(s) 'HateD Applied' do not appear together in any document.


Tokenized Query: ['ca', "n't"]
Congratulations! The word(s) 'Can't' appear together in the following document ID(s): {0}
Document ID: 0, Document Name: The Young Man in Business by Edward William Bok


Tokenized Query: ['did', "n't"]
Congratulations! The word(s) 'didn't' appear together in the following document ID(s): {0}
Document ID: 0, Document Name: The Young Man in Business by Edward William Bok


Tokenized Query: ['state-of-the-art']
I'm sorry, the word(s) 'state-of-the-art' do not appear together in any document.


Tokenized Query: ['elliott-fisher']
I'm sorry, the word(s) 'Elliott-Fisher' do not appear together in any document.


Tokenized Query: ['mr', '.']
Congrat

,word,document_ids
0,the,"0,1"
1,project,"0,1"
2,gutenberg,"0,1"
3,ebook,"0,1"
4,of,"0,1"
...,...,...
7351,66,1
7352,whifflles,1
7353,109,1
7354,wilyums,1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected 

The Project Gutenberg eBook of The Young Man in Business
[END of paragraph]
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.
[END of paragraph]
Title: The Young Man in Business
[END of paragraph]
Author: Edward William Bok
[END of paragraph]
Release date: March 3, 2010 [eBook #31494]
                Most recently updated: January 6, 2021
[END of paragraph]
Language: English
[END of paragraph]
Credits: Produced by Chuck Greif (from files available at www.archive.org)
[END of paragraph]
*** START OF THE PROJECT GUTENBERG EBOOK THE YOUNG MAN IN BUSINESS ***
[END of paragraph]
Produced by Chuck Greif (f

In [ ]:
# Add embeddings to the dataset with truncation
max_length = 512

def add_embeddings(example):
    inputs = ctx_tokenizer(example["content"], truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
    embeddings = ctx_encoder(**inputs).pooler_output.squeeze().numpy()
    return {"embeddings": embeddings}

ds_with_embeddings = ds.map(add_embeddings)
ds_with_embeddings.add_faiss_index(column='embeddings')

Map:   0%|          | 0/1097 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['doc_id', 'doc_title', 'para_id', 'content', 'embeddings'],
    num_rows: 1097
})

In [ ]:
# Load the DPR question encoder and tokenizer
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Example query
question = "How to make money on Wall Street?"
inputs = q_tokenizer(question, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
question_embedding = q_encoder(**inputs).pooler_output.squeeze().numpy()

# Retrieve nearest examples using FAISS
scores, retrieved_examples = ds_with_embeddings.get_nearest_examples('embeddings', question_embedding, k=10)

# Extract predicted relevance scores from retrieved examples
predicted_scores = {}
for i in range(len(retrieved_examples["doc_id"])):
    doc_id = retrieved_examples["doc_id"][i]
    para_id = retrieved_examples["para_id"][i]
    score = scores[i]
    if predicted_scores.get("Example Query") is None:
        predicted_scores["Example Query"] = []
    predicted_scores["Example Query"].append((doc_id, para_id, score))


# Print predicted_scores
print("Predicted Scores:")
for query, scores in predicted_scores.items():
    print(f"Query: {query}")
    for doc_id, para_id, score in scores:
        print(f"  Document ID: {doc_id}, Paragraph ID: {para_id}, Score: {score}")



Predicted Scores:
Query: Example Query
  Document ID: 1, Paragraph ID: 713, Score: 82.16383361816406
  Document ID: 1, Paragraph ID: 113, Score: 82.87889862060547
  Document ID: 1, Paragraph ID: 710, Score: 83.32301330566406
  Document ID: 1, Paragraph ID: 301, Score: 85.00048065185547
  Document ID: 1, Paragraph ID: 458, Score: 86.56836700439453
  Document ID: 1, Paragraph ID: 148, Score: 86.65271759033203
  Document ID: 0, Paragraph ID: 41, Score: 87.58847045898438
  Document ID: 1, Paragraph ID: 37, Score: 87.74506378173828
  Document ID: 1, Paragraph ID: 727, Score: 88.2590560913086
  Document ID: 1, Paragraph ID: 832, Score: 88.85917663574219


In [ ]:
# Display the retrieved paragraphs and their document titles
for i in range(len(retrieved_examples["content"])):
    print(f"Document Title: {retrieved_examples['doc_title'][i]}")
    print(f"Paragraph: {retrieved_examples['content'][i]}\n")

Document Title: Dumbells of Business by Louis Custer Martin Reed
Paragraph: But gradually they got used to the upset and flurry, and when the
monthly Balance Sheet began to smile and then to grin, it poked their
Ambition in the ribs and the first thing they knew they were actually
craning their shaved necks for business in the Domestic as well as in
the Export arena.

Document Title: Dumbells of Business by Louis Custer Martin Reed
Paragraph: If the Credit Man lost only 1/40th of 1 per cent on the year’s
accounts, he would dodge around in front of the throne expecting to be
caught and laureled.

Document Title: Dumbells of Business by Louis Custer Martin Reed
Paragraph: The manufactory in which he hopped about and drew his Dollar and a
Quarter a week, was not so big as Standard Oil, but it had a side-track
and a time-clock and involved the activities of seventy-five men,
two of which were old women—the Sales & Advertising Manager and the
Proprietor.

Document Title: Dumbells of Busines

In [ ]:
# Access the FAISS index
faiss_index = ds_with_embeddings.get_index('embeddings').faiss_index

# Use the FAISS index for range search
limits, distances, indices = faiss_index.range_search(x=question_embedding.reshape(1, -1), thresh=0.95)

# Save FAISS index to Google Drive
faiss_index_path = '/content/drive/My Drive/Documents/my_index.faiss'
ds_with_embeddings.save_faiss_index('embeddings', faiss_index_path)

# Save dataset with embeddings
dataset_with_embeddings_path = '/content/drive/My Drive/Documents/dataset_with_embeddings.pt'
torch.save(ds_with_embeddings, dataset_with_embeddings_path)

# Save and load FAISS index from Google Drive
ds = Dataset.from_dict({
    "doc_id": [para["doc_id"] for para in paragraphs],
    "doc_title": [para["doc_title"] for para in paragraphs],
    "para_id": [para["para_id"] for para in paragraphs],
    "content": [para["content"] for para in paragraphs]
})
ds.load_faiss_index('embeddings', faiss_index_path)


In [ ]:
import numpy as np
import os

# Function to export embeddings and metadata for TensorFlow Projector
def export_to_tsv(dataset, output_dir):
    embeddings_path = os.path.join(output_dir, 'embeddings.tsv')
    metadata_path = os.path.join(output_dir, 'metadata.tsv')

    with open(embeddings_path, 'w', encoding='utf-8') as embeddings_file, \
         open(metadata_path, 'w', encoding='utf-8') as metadata_file:

        # Write headers
        metadata_file.write("doc_id\tdoc_title\tpara_id\tcontent\n")

        for example in dataset:
            if 'embeddings' in example and example['embeddings'] is not None:
                # Write embeddings
                embeddings_file.write('\t'.join(map(str, example['embeddings'])) + '\n')
                # Write metadata
                content_snippet = example['content'][:30].replace('\n', ' ') + '...'
                metadata_file.write(f"{example['doc_id']}\t{example['doc_title']}\t{example['para_id']}\t{content_snippet}\n")

# Create output directory
export_dir = '/content/drive/My Drive/Documents/tensorflow_projector'
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

# Export embeddings and metadata to the specified directory
export_to_tsv(ds_with_embeddings, export_dir)

print(f"Embeddings and metadata exported to {export_dir} for TensorFlow Projector.")


Embeddings and metadata exported to /content/drive/My Drive/Documents/tensorflow_projector for TensorFlow Projector.


GROUND_TRUTH

In [ ]:
from sklearn.metrics import ndcg_score

# Example query
query = "How to make money on Wall Street?"

# Tokenize and encode the query using the DPR question encoder
inputs = q_tokenizer(query, truncation=True, padding="max_length", max_length=max_length, return_tensors="pt")
question_embedding = q_encoder(**inputs).pooler_output.squeeze().numpy()

# Retrieve nearest examples using FAISS
k = 10  # Number of top documents to retrieve
scores, retrieved_examples = ds_with_embeddings.get_nearest_examples('embeddings', question_embedding, k=k)

# Print the retrieved paragraphs and their document titles
for i in range(len(retrieved_examples["content"])):
    print(f"Document Title: {retrieved_examples['doc_title'][i]}")
    print(f"Paragraph: {retrieved_examples['content'][i]}\n")

# Example ground-truth relevance scores for the retrieved paragraphs
# (You need to define these based on your ground-truth data)
true_relevance = np.asarray([[3, 3, 2, 2, 1, 2, 0, 1, 2, 3]])

# Predicted relevance scores (similarity scores from FAISS retrieval)
predicted_scores = np.asarray([scores])

# Calculate nDCG
ndcg_at_k = ndcg_score(true_relevance, predicted_scores, k=k)
print(f'nDCG@{k}:', ndcg_at_k)


Document Title: Dumbells of Business by Louis Custer Martin Reed
Paragraph: But gradually they got used to the upset and flurry, and when the
monthly Balance Sheet began to smile and then to grin, it poked their
Ambition in the ribs and the first thing they knew they were actually
craning their shaved necks for business in the Domestic as well as in
the Export arena.

Document Title: Dumbells of Business by Louis Custer Martin Reed
Paragraph: If the Credit Man lost only 1/40th of 1 per cent on the year’s
accounts, he would dodge around in front of the throne expecting to be
caught and laureled.

Document Title: Dumbells of Business by Louis Custer Martin Reed
Paragraph: The manufactory in which he hopped about and drew his Dollar and a
Quarter a week, was not so big as Standard Oil, but it had a side-track
and a time-clock and involved the activities of seventy-five men,
two of which were old women—the Sales & Advertising Manager and the
Proprietor.

Document Title: Dumbells of Busines

In [ ]:
# Print the relevance scores and the retrieved paragraphs
print("Relevance Scores:")
print(true_relevance)

print("\nPredicted Scores:")
for i in range(len(retrieved_examples["content"])):
    doc_id = retrieved_examples['doc_id'][i]
    para_id = retrieved_examples['para_id'][i]
    score = scores[i]
    print(f"Document ID: {doc_id}, Paragraph ID: {para_id}, Score: {score}")

print("\nRetrieved Paragraphs:")
for i in range(len(retrieved_examples["content"])):
    print(f"Document Title: {retrieved_examples['doc_title'][i]}")
    print(f"Paragraph ID: {retrieved_examples['para_id'][i]}")
    print(f"Paragraph: {retrieved_examples['content'][i]}")

Relevance Scores:
[[3 3 2 2 1 2 0 1 2 3]]

Predicted Scores:
Document ID: 1, Paragraph ID: 713, Score: 82.16383361816406
Document ID: 1, Paragraph ID: 113, Score: 82.87889862060547
Document ID: 1, Paragraph ID: 710, Score: 83.32301330566406
Document ID: 1, Paragraph ID: 301, Score: 85.00048065185547
Document ID: 1, Paragraph ID: 458, Score: 86.56836700439453
Document ID: 1, Paragraph ID: 148, Score: 86.65271759033203
Document ID: 0, Paragraph ID: 41, Score: 87.58847045898438
Document ID: 1, Paragraph ID: 37, Score: 87.74506378173828
Document ID: 1, Paragraph ID: 727, Score: 88.2590560913086
Document ID: 1, Paragraph ID: 832, Score: 88.85917663574219

Retrieved Paragraphs:
Document Title: Dumbells of Business by Louis Custer Martin Reed
Paragraph ID: 713
Paragraph: But gradually they got used to the upset and flurry, and when the
monthly Balance Sheet began to smile and then to grin, it poked their
Ambition in the ribs and the first thing they knew they were actually
craning their shave

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit
import streamlit as st

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [ ]:
%%writefile /content/drive/MyDrive/search_engine.py

import sqlite3
from nltk.tokenize import word_tokenize
from collections import defaultdict
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

class InvertedIndex:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS inverted_index (
                                word TEXT PRIMARY KEY,
                                document_ids TEXT
                            )''')

    def save_index(self, inverted_index):
        for word, document_ids in inverted_index.items():
            doc_ids_str = ','.join(str(doc_id) for doc_id in document_ids)
            self.cursor.execute("INSERT INTO inverted_index (word, document_ids) VALUES (?, ?)", (word, doc_ids_str))
        self.conn.commit()

    def close_connection(self):
        self.conn.close()

class DocumentTokenizer:
    def __init__(self, documents):
        self.documents = documents

    def tokenize(self):
        tokenized_docs = {}
        for documentID, document_content in enumerate(self.documents):
            try:
                tokens = word_tokenize(document_content)
                tokenized_docs[documentID] = tokens
            except Exception as e:
                print(f"Error processing document {documentID}: {e}")
        return tokenized_docs

class InvertedIndexBuilder:
    def __init__(self, documents):
        self.documents = documents
        self.inverted_index = defaultdict(set)

    def build_index(self):
        for documentID, tokens in self.documents.items():
            for word in tokens:
                self.inverted_index[word.lower()].add(documentID)

    def get_index(self):
        return self.inverted_index

class SearchEngine:
    def __init__(self, index_db_path, documents_mapping, dataset_with_embeddings):
        self.index_db_path = index_db_path
        self.documents_mapping = documents_mapping
        self.ds_with_embeddings = dataset_with_embeddings

    def lookup_word(self, word):
        conn = sqlite3.connect(self.index_db_path)
        cursor = conn.cursor()
        cursor.execute("SELECT document_ids FROM inverted_index WHERE word=?", (word,))
        result = cursor.fetchone()
        conn.close()

        if result:
            document_ids = set(map(int, result[0].split(',')))
            return document_ids
        else:
            return set()

    def search(self, query):
        # Compute the query embedding
        inputs = q_tokenizer(query, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
        with torch.no_grad():
            question_embedding = q_encoder(**inputs).pooler_output.squeeze().numpy()

        # Retrieve nearest examples using FAISS
        scores, retrieved_examples = self.ds_with_embeddings.get_nearest_examples('embeddings', question_embedding, k=10)

        search_results = []
        for i in range(len(retrieved_examples["content"])):
            doc_id = retrieved_examples["doc_id"][i]
            doc_title = retrieved_examples["doc_title"][i]
            paragraph = retrieved_examples["content"][i]
            score = scores[i]
            search_results.append((doc_id, doc_title, paragraph, score))

        return search_results

# Initialize the question encoder and tokenizer
q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

# Define the function to get query embedding
def get_query_embedding(query):
    inputs = q_tokenizer(query, truncation=True, padding="max_length", max_length=512, return_tensors="pt")
    with torch.no_grad():
        embedding = q_encoder(**inputs).pooler_output.squeeze().numpy()
    return embedding

Writing /content/drive/MyDrive/search_engine.py


In [ ]:
# Check if the file exists
print(os.path.exists('/content/drive/MyDrive/search_engine.py'))  # Should print: True

# Display the contents of the file
!cat /content/drive/MyDrive/search_engine.py


True

import sqlite3
from nltk.tokenize import word_tokenize
from collections import defaultdict
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

class InvertedIndex:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS inverted_index (
                                word TEXT PRIMARY KEY,
                                document_ids TEXT
                            )''')

    def save_index(self, inverted_index):
        for word, document_ids in inverted_index.items():
            doc_ids_str = ','.join(str(doc_id) for doc_id in document_ids)
            self.cursor.execute("INSERT INTO inverted_index (word, document_ids) VALUES (?, ?)", (word, doc_ids_str))
        self.conn.commit()

    def close_connection(self):
        self.co

In [ ]:
import sys

# Add the directory containing search_engine.py to sys.path
sys.path.append('/content/drive/MyDrive')

# Verify sys.path
print(sys.path)


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive']


In [ ]:
# Display the contents of search_engine.py
with open('/content/drive/MyDrive/search_engine.py', 'r') as file:
    print(file.read())



import sqlite3
from nltk.tokenize import word_tokenize
from collections import defaultdict
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer
import torch

class InvertedIndex:
    def __init__(self, db_path):
        self.db_path = db_path
        self.conn = sqlite3.connect(self.db_path)
        self.cursor = self.conn.cursor()
        self.create_table()

    def create_table(self):
        self.cursor.execute('''CREATE TABLE IF NOT EXISTS inverted_index (
                                word TEXT PRIMARY KEY,
                                document_ids TEXT
                            )''')

    def save_index(self, inverted_index):
        for word, document_ids in inverted_index.items():
            doc_ids_str = ','.join(str(doc_id) for doc_id in document_ids)
            self.cursor.execute("INSERT INTO inverted_index (word, document_ids) VALUES (?, ?)", (word, doc_ids_str))
        self.conn.commit()

    def close_connection(self):
        self.conn.cl

In [ ]:
#!pip install streamlit --quiet

In [ ]:
%%writefile /content/drive/MyDrive/app.py
import importlib
#!pip install streamlit --quiet
#!pip install streamlit
import streamlit as st
import torch
import search_engine
importlib.reload(search_engine)

from search_engine import SearchEngine

# Initialize your search engine
index_db_path = '/content/drive/MyDrive/Documents/inverted_index.db'
document_mapping = {0: 'Document 1', 1: 'Document 2'}  # Example mapping

# Load dataset with embeddings
dataset_with_embeddings_path = '/content/drive/My Drive/Documents/dataset_with_embeddings.pt'
ds_with_embeddings = torch.load(dataset_with_embeddings_path)

search_engine = SearchEngine(index_db_path, document_mapping, ds_with_embeddings)

# Streamlit App
st.title("Document Search Engine")

# Input query from the user
query = st.text_input("Enter your search query:")

# Search button
if st.button("Search"):
    # Perform search
    search_results = search_engine.search(query)

    if search_results:
        # Sort search results by score in ascending order
        search_results.sort(key=lambda x: x[3], reverse=True)

        # Display top 10 search results
        st.subheader("Top 10 Search Results:")
        for rank, result in enumerate(search_results, start=1):
            doc_id = result[0]
            doc_title = result[1]
            paragraph = result[2]
            score = result[3]
            st.write(f"Rank: {rank}")
            st.write(f"Document: {doc_title} (ID: {doc_id})")
            st.write(f"Paragraph: {paragraph}")
            st.write(f"Score: {score}")
            st.write("---")
    else:
        st.write("No results found for the query.")

Overwriting /content/drive/MyDrive/app.py


In [ ]:
# Install localtunnel
!npm install -g localtunnel

# Run Streamlit app in the background
!streamlit run /content/drive/MyDrive/app.py & npx localtunnel --port 8501

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.444s



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.82.157.30:8501

npx: installed 22 in 3.265s
your url is: https://bumpy-tools-relate.loca.lt
Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a 

In [ ]:
# Install localtunnel
!npm install -g localtunnel

# Run Streamlit app in the background
!streamlit run /content/drive/MyDrive/app.py &

# Use localtunnel to expose the Streamlit app
!lt --port 8501


/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 1.296s



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.142.244.185:8501

  Stopping...
your url is: https://gentle-heads-tickle.loca.lt
